In [ ]:
#   Author: Alexis Hilts
#   Date:   23rd April, 2025

#   Script Purpose: This script quantifies the neurite density across the z-slices of images taken of growth 
#   through the gel interface. You must manually select an ROI for each side of the gel interface where the 
#   growth is. You can do this before on imageJ using the ‘delete_outside_ROI’ macro on the Teams or you can 
#   do it in the script.

# Import necessary libraries
import numpy as np                       
import cv2                               
from skimage.morphology import remove_small_objects  
from tifffile import imread              
from matplotlib import pyplot as plt     
from matplotlib.patches import Rectangle 

# Define the file path to the TIFF image
filepath: str = r"GI_Images_poster/empty_channel.tif" # write in your file path here (will have to manually change for each image)
im_arr = imread(filepath)                # Read the 3D image stack as a NumPy array (shape: Z x Y x X)

# The following ROIs must be adjusted manually

# Define GelMA ROI
X_ROI: list = [1500, 2000]                    # X-coordinate range of the ROI
Y_ROI: list = [600, 1200]                     # Y-coordinate range of the ROI
Z_LEVEL: int = -1                             # Z-plane to visualize (-1 is the last slice where neurites usually grow in GelMA)

# Define Ultimatrix ROI with the same dimensions as GelMA ROI
X_ROI_U: list = [1200, 1800]
Y_ROI_U: list = [100, 700]
Z_LEVEL: int = -10

# Enhance contrast using cv2.addWeighted 
im_arr = cv2.addWeighted(im_arr, 10, np.zeros(im_arr.shape, im_arr.dtype), 0, 100)
im_arr = im_arr[:-10,:,:]


# Plot the selected Z-plane with ROIs overlaid
fig, ax = plt.subplots(figsize=(8, 8), dpi=120)
plt.imshow(im_arr[Z_LEVEL, :, :])             # Display the Z-plane as a grayscale image
rect_g = Rectangle((X_ROI[0], Y_ROI[0]), X_ROI[1] - X_ROI[0], Y_ROI[1] - Y_ROI[0], alpha=0.1, edgecolor='darkred', facecolor='red')
rect_u = Rectangle((X_ROI_U[0], Y_ROI_U[0]), X_ROI_U[1] - X_ROI_U[0], Y_ROI_U[1] - Y_ROI_U[0], alpha=0.1, edgecolor='blue', facecolor='blue')
ax.add_patch(rect_g)                          # Add GelMA ROI as a red rectangle
ax.add_patch(rect_u)                          # Add Ultimatrix ROI as a blue rectangle
plt.ylabel('Y Position')
plt.xlabel('X Position')
plt.grid(False)
plt.gray()
plt.show()

# Extract the ROIs from the full image stack
g_im_arr = im_arr[:, Y_ROI[0]:Y_ROI[1], X_ROI[0]:X_ROI[1]]  # GelMA ROI
u_im_arr = im_arr[:, Y_ROI_U[0]:Y_ROI_U[1], X_ROI_U[0]:X_ROI_U[1]]  # Ultimatrix ROI

# Initialize empty lists to store neurite density for each Z-plane
neurite_density_g = []                       # For GelMA ROI
neurite_density_u = []                       # For Ultimatrix ROI
z_vals = [] # for plotting

# Loop through each Z-plane in the image stack
for i in range(im_arr.shape[0]): 
    z_vals.append(i)
    # Select the current Z-plane for both ROIs
    g_img = g_im_arr[i, :, :].astype(np.float32)  # GelMA ROI
    u_img = u_im_arr[i, :, :].astype(np.float32)  # Ultimatrix ROI

    # Apply a wide Gaussian blur to estimate the background
    g_bkg_Gaussian = cv2.GaussianBlur(g_img, (41, 41), 10)
    u_bkg_Gaussian = cv2.GaussianBlur(u_img, (41, 41), 10)

    # Subtract the background
    g_bkg_subtracted = g_img - g_bkg_Gaussian
    u_bkg_subtracted = u_img - u_bkg_Gaussian
    g_bkg_subtracted[g_bkg_subtracted < 0] = 0  # Clip negative values to 0
    u_bkg_subtracted[u_bkg_subtracted < 0] = 0

    # Apply a smaller Gaussian blur for smoothing
    g_img_smoothed = cv2.GaussianBlur(g_bkg_subtracted, (5, 5), 2)
    u_img_smoothed = cv2.GaussianBlur(u_bkg_subtracted, (5, 5), 2)

    # Binarize the images by keeping only the top 20% brightest pixels
    g_binary_img = g_bkg_subtracted > np.percentile(g_bkg_subtracted, 80)
    u_binary_img = u_bkg_subtracted > np.percentile(u_bkg_subtracted, 80)

    # Remove small objects to clean up the binary image
    g_binary_img_cleaned = remove_small_objects(g_binary_img, min_size=100)
    u_binary_img_cleaned = remove_small_objects(u_binary_img, min_size=100)

    # Calculate neurite density for each ROI
    g_neurite_density = np.sum(g_binary_img_cleaned) / g_binary_img_cleaned.size
    u_neurite_density = np.sum(u_binary_img_cleaned) / u_binary_img_cleaned.size

    # Append the calculated densities to the respective lists
    neurite_density_g.append(g_neurite_density)
    neurite_density_u.append(u_neurite_density)

x = np.array(z_vals)
x = x*0.015
# Plot neurite density across all Z-planes
fig, ax = plt.subplots(figsize=(8, 8), dpi=300)
plt.plot(x, neurite_density_g, 'o-', color = 'firebrick', linewidth = 5, markersize = 8) #label='GelMA')   # Plot GelMA density
plt.ylim(0,0.2)
#plt.plot(neurite_density_u, label='Ultimatrix')  # Plot Ultimatrix density
plt.title('Empty Channel', fontsize = 20) # (GelMA vs Ultimatrix)')
plt.xlabel('Z-distance through sample (mm)', fontsize = 14)
plt.ylabel('Neurite Density', fontsize = 14)
#plt.legend()
plt.show()

# Store results for further analysis - e.g. plotting all densities together
gelma_nc2_3 = neurite_density_g
ulti_nc2_3 = neurite_density_u


In [ ]:
# this script is modified to analyze pre-selected ROIs of the channel and the well to compare neurite density across the z-stack

# Import necessary libraries
import os
import numpy as np                       
import cv2                               
from skimage.morphology import remove_small_objects  
from tifffile import imread              
from matplotlib import pyplot as plt    
from matplotlib.patches import Rectangle

# Define the file path to the TIFF image
filepath: str = r"20241209_TC_IL_GI_stacks/ulti-G50-GSH/DRG3_channel.tif" # write in your file path here (will have to manually change for each image)
im_arr = imread(filepath)                # Read the 3D image stack as a NumPy array (shape: Z x Y x X)
im_arr = cv2.addWeighted(im_arr, 10, np.zeros(im_arr.shape, im_arr.dtype), 0, 100)
fig, ax = plt.subplots(figsize=(5,5), dpi=120)
plt.gray()
plt.imshow(im_arr[47,:,:])
plt.ylabel('Y Position')
plt.xlabel('X Position')
plt.show()



# Initialize empty lists to store neurite density for each Z-plane
neurite_density = []                       

# Loop through each Z-plane in the image stack
for i in range(im_arr.shape[0]): 
    # Select the current Z-plane for both ROIs
    img = im_arr[i, :, :].astype(np.float32) 

    # Apply a wide Gaussian blur to estimate the background
    bkg_Gaussian = cv2.GaussianBlur(img, (71, 71), 10)
    # if i == 75:
    #     fig, ax = plt.subplots(figsize=(5,5), dpi=120)
    #     plt.gray()
    #     plt.imshow(bkg_Gaussian)
    #     plt.ylabel('Y Position')
    #     plt.xlabel('X Position')
    #     plt.show()

    # Subtract the background
    bkg_subtracted = img - bkg_Gaussian
    bkg_subtracted[bkg_subtracted < 0] = 0  # Clip negative values to 0

    # Apply a smaller Gaussian blur for smoothing
    img_smoothed = cv2.GaussianBlur(bkg_subtracted, (5, 5), 2)
    # if i == 75:
    #     fig, ax = plt.subplots(figsize=(5,5), dpi=120)
    #     plt.gray()
    #     plt.imshow(img_smoothed)
    #     plt.ylabel('Y Position')
    #     plt.xlabel('X Position')
    #     plt.show()

    # Binarize the images by keeping only the top 20% brightest pixels
    binary_img = bkg_subtracted > np.percentile(bkg_subtracted, 95)
    # if i == 75:
    #     fig, ax = plt.subplots(figsize=(5,5), dpi=120)
    #     plt.gray()
    #     plt.imshow(binary_img)
    #     plt.ylabel('Y Position')
    #     plt.xlabel('X Position')
    #     plt.show()

    # Remove small objects to clean up the binary image
    binary_img_cleaned = remove_small_objects(binary_img, min_size=100)
    if i == 45:
        fig, ax = plt.subplots(figsize=(5,5), dpi=120)
        plt.gray()
        plt.imshow(binary_img_cleaned)
        plt.ylabel('Y Position')
        plt.xlabel('X Position')
        plt.show()

    # Calculate neurite density for each ROI
    neurite_density_temp = np.sum(binary_img_cleaned) / binary_img_cleaned.size

    # Append the calculated densities to the respective lists
    neurite_density.append(neurite_density_temp)

# Plot neurite density across all Z-planes
x = np.arange(0,len(neurite_density),1)
plt.plot(x, neurite_density, '.-')   # Plot GelMA density
plt.title('Neurite Density Across Z-planes (GelMa vs Ultimatrix)')
plt.xlabel('Z-plane')
plt.ylabel('Neurite Density')
#plt.legend()
plt.show()
print(neurite_density)

# Store results for further analysis - e.g. plotting all densities together
# gelma_nc2_3 = neurite_density_g
# ulti_nc2_3 = neurite_density_u


In [ ]:
# This cell can be used to compare the well and channel of the same image, as long as the ROIs have been selected and saved
# as 2 separate images before
import os
import numpy as np
import cv2
from skimage.morphology import remove_small_objects
from tifffile import imread
import matplotlib.pyplot as plt

# Function to process TIFF images and calculate neurite density
def process_image(filepath):
    im_arr = imread(filepath)
    im_arr = cv2.addWeighted(im_arr, 10, np.zeros(im_arr.shape, im_arr.dtype), 0, 100)

    neurite_density = []
    for i in range(im_arr.shape[0]):
        img = im_arr[i, :, :].astype(np.float32)
        bkg_Gaussian = cv2.GaussianBlur(img, (71, 71), 10)
        bkg_subtracted = img - bkg_Gaussian
        bkg_subtracted[bkg_subtracted < 0] = 0
        img_smoothed = cv2.GaussianBlur(bkg_subtracted, (5, 5), 2)
        binary_img = img_smoothed > np.percentile(img_smoothed, 95)
        binary_img_cleaned = remove_small_objects(binary_img, min_size=100)
        if i == 75:
            fig, ax = plt.subplots(figsize=(5,5), dpi=120)
            plt.gray()
            plt.imshow(binary_img_cleaned)
            plt.ylabel('Y Position')
            plt.xlabel('X Position')
            plt.show()
        neurite_density_temp = np.sum(binary_img_cleaned) / binary_img_cleaned.size
        neurite_density.append(neurite_density_temp)

    return neurite_density

# File paths
channel_filepath = "20241209_TC_IL_GI_stacks/ulti-G50-GSH/DRG3_channel.tif"
well_filepath = "20241209_TC_IL_GI_stacks/ulti-G50-GSH/DRG3_well.tif"

# Process images
channel_density = process_image(channel_filepath)
well_density = process_image(well_filepath)

# Plotting
plt.figure(figsize=(10, 6))
z_planes = np.arange(len(channel_density)) * 15 / 1000  # Convert to mm

plt.plot(z_planes, channel_density, label='Channel')
plt.plot(z_planes, well_density, label='Well')

plt.xlabel('Z Position (mm)')
plt.ylabel('Neurite Density')
plt.title('Neurite Density Across Z-planes')
plt.legend()
plt.grid(True)
plt.gca().set_facecolor('white')  # Set background to white
plt.savefig('ulti-50DoF-GSH-DRG3_plot.png', bbox_inches='tight', facecolor='white')
plt.show()

